# Import networkx json and save them in our data format

In [ ]:
from pathlib import Path
import pandas as pd
import json
import zarr
import os
import sequitur as sq
import networkx as nx

def list_files(path):
    startpath = str(path.absolute())
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            if level < 2 or f.endswith('parquet'):
                print('{}{}'.format(subindent, f))

## Import images and labels

In [ ]:
root = Path('examples')

zarr_file = zarr.open(Path(root, 'example_data', 'Fluo-N3DH-CHO.zarr'))
images = zarr_file['images'][:]
labels = zarr_file['segmentations'][:]
print(f'Got images of shape {images.shape}')

## Import nodes and edges

In [ ]:
# Read networkx exported json
nodes, edges = None, None
with open(Path(root, 'example_data', 'candidate_graph.json'), 'r') as f:
    candidate_graph = json.load(f)

    # format nodes and edges
    nodes  = pd.DataFrame(candidate_graph['nodes'])
    nodes.rename(columns={'id': 'node_id'}, inplace=True)

    edges = pd.DataFrame(candidate_graph['links'])
    edges['edge_id'] = [i for i in range(edges.shape[0])]
    edges.rename(columns={'source': 'source_id', 'target': 'target_id'}, inplace=True)


In [ ]:
print(f'Got {len(nodes)} entries')
nodes.head()

In [ ]:
edges.head()

## Write to data format

In [8]:
path_to_file = Path(root, 'example.sequitur')

myfile = sq.open(path_to_file, mode='w')
list_files(path_to_file.absolute())

example.sequitur/


In [9]:
# Write images
myfile.write_image(images)
myfile.write_annotations(labels)

list_files(path_to_file.absolute())

example.sequitur/
    images.zarr/
        .zgroup
        images/
        annotations/


In [10]:
# Write nodes and edges
myfile.write_nodes(nodes)
myfile.write_edges(edges)

list_files(path_to_file.absolute())

example.sequitur/
    graph/
        nodes/
            nodes.parquet
        edges/
            edges.parquet
    images.zarr/
        .zgroup
        images/
        annotations/


# Back import into networkx

In [11]:
read_nodes = myfile.read_nodes()
read_nodes.rename(columns={'node_id': 'id'}, inplace=True)
read_edges = myfile.read_edges()
read_edges.rename(columns={'source_id': 'source', 'target_id': 'target'}, inplace=True)

In [12]:

# create dict similar to networkx export
networkx_format = {
    'directed': 'true', 
    'multigraph': 'false', 
    'graph': {}, 
    'nodes': read_nodes.to_dict(orient='records'),
    'links': read_edges.to_dict(orient='records'),
}
networkx_format

{'directed': 'true',
 'multigraph': 'false',
 'graph': {},
 'nodes': [{'id': 0,
   't': 0,
   'x': 0.0,
   'y': 55.0,
   'z': 55.0,
   'score': 1,
   'GT_ID': 1},
  {'id': 1, 't': 1, 'x': 0.0, 'y': 59.0, 'z': 55.5, 'score': 1, 'GT_ID': 1},
  {'id': 2, 't': 2, 'x': 0.0, 'y': 53.0, 'z': 55.0, 'score': 1, 'GT_ID': 1},
  {'id': 3, 't': 3, 'x': 0.0, 'y': 60.0, 'z': 56.0, 'score': 1, 'GT_ID': 1},
  {'id': 4, 't': 4, 'x': 0.0, 'y': 61.0, 'z': 56.5, 'score': 1, 'GT_ID': 1},
  {'id': 5, 't': 5, 'x': 0.0, 'y': 71.0, 'z': 55.0, 'score': 1, 'GT_ID': 1},
  {'id': 6, 't': 6, 'x': 0.0, 'y': 71.0, 'z': 55.5, 'score': 1, 'GT_ID': 1},
  {'id': 7, 't': 7, 'x': 0.0, 'y': 71.0, 'z': 56.0, 'score': 1, 'GT_ID': 1},
  {'id': 8, 't': 8, 'x': 0.0, 'y': 61.0, 'z': 53.0, 'score': 1, 'GT_ID': 1},
  {'id': 9, 't': 9, 'x': 0.0, 'y': 60.0, 'z': 52.0, 'score': 1, 'GT_ID': 1},
  {'id': 10, 't': 10, 'x': 0.0, 'y': 60.0, 'z': 58.0, 'score': 1, 'GT_ID': 1},
  {'id': 11, 't': 11, 'x': 0.0, 'y': 77.0, 'z': 53.0, 'score': 1,

In [13]:
from networkx import json_graph
g = json_graph.node_link_graph(networkx_format)
print(f'Graph with {g.number_of_nodes()} nodes and {g.number_of_edges()} edges')

Graph with 905 nodes and 9092 edges
